In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['GOOGLE_API_KEY'] = os.getenv("GOOGLE_API_KEY")
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [2]:
### Data Ingestion -- From the website we need to Scrape the data

from langchain_community .document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
loader = WebBaseLoader("https://pypi.org/project/langchain-google-genai/")
loader

In [5]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://pypi.org/project/langchain-google-genai/', 'title': 'langchain-google-genai · PyPI', 'description': "An integration package connecting Google's genai package and LangChain", 'language': 'en'}, page_content='\n\n\n\n\n\n\n\n\nlangchain-google-genai · PyPI\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content\n\nSwitch to mobile version    \n\n\n\n\n\n\n\nWarning\n\nSome features may not work without JavaScript. Please try enabling it if you encounter problems.\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\nSearch PyPI\n\n\n\nSearch\n\n\n\n \n\n\nHelp\nSponsors\nLog in\nRegister\n\n\n\n\nMenu      \n\n\n\n\nHelp\nSponsors\nLog in\nRegister\n\n\n\n \n\n\n\n\nSearch PyPI\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n        langchain-google-genai 1.0.10\n      \n\n\npip install langchain-google-genai\n\n\nCopy PIP instructions\n\n\n\n\n\n\nLatest version\n\n\nReleased: \n  Aug 21, 2024\n \n\n\n\n\n\n \nAn integration package connecting Google\'s genai 

In [6]:
### Load Data --> Docs --> Divide our text into chunks -->text -->Vectors -->vector Embeddings --> Vector Stores
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
documents = text_splitter.split_documents(docs)
documents

[Document(metadata={'source': 'https://pypi.org/project/langchain-google-genai/', 'title': 'langchain-google-genai · PyPI', 'description': "An integration package connecting Google's genai package and LangChain", 'language': 'en'}, page_content="langchain-google-genai · PyPI\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content\n\nSwitch to mobile version    \n\n\n\n\n\n\n\nWarning\n\nSome features may not work without JavaScript. Please try enabling it if you encounter problems.\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\nSearch PyPI\n\n\n\nSearch\n\n\n\n \n\n\nHelp\nSponsors\nLog in\nRegister\n\n\n\n\nMenu      \n\n\n\n\nHelp\nSponsors\nLog in\nRegister\n\n\n\n \n\n\n\n\nSearch PyPI\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n        langchain-google-genai 1.0.10\n      \n\n\npip install langchain-google-genai\n\n\nCopy PIP instructions\n\n\n\n\n\n\nLatest version\n\n\nReleased: \n  Aug 21, 2024\n \n\n\n\n\n\n \nAn integration package connecting Google's genai package and LangCha

In [9]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [11]:
from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(documents,embeddings)
vectorstoredb

In [12]:
query =  "How to Do embedding using Google Generative AI"
result = vectorstoredb.similarity_search(query)
result[0].page_content

'A public image URL\nAn accessible gcs file (e.g., "gcs://path/to/file.png")\nA local file path\nA base64 encoded image (e.g., data:image/png;base64,abcd124)\nA PIL image\n\nEmbeddings\nThis package also adds support for google\'s embeddings models.\nfrom langchain_google_genai import GoogleGenerativeAIEmbeddings\n\nembeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")\nembeddings.embed_query("hello, world!")\n\nSemantic Retrieval\nEnables retrieval augmented generation (RAG) in your application.\n# Create a new store for housing your documents.\ncorpus_store = GoogleVectorStore.create_corpus(display_name="My Corpus")\n\n# Create a new document under the above corpus.\ndocument_store = GoogleVectorStore.create_document(\n    corpus_id=corpus_store.corpus_id, display_name="My Document"\n)'

In [15]:
###Retrival Chain,Document Chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
Answer The Following Question based Only on the Provided Context:
<context>
{context}
</context>
"""
)


document_chain =  create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer The Following Question based Only on the Provided Context:\n<context>\n{context}\n</context>\n'))])
| ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001EFE5506930>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x000001EFE553F740>, default_metadata=())
| StrOutputParser(), config={'run_name': 'stuff_documents_chain'})

In [ ]:
document_chain.invoke({
    "input":""
})